In [1]:
import os

if os.path.exists( ".env" ):
    from dotenv import load_dotenv
    load_dotenv()

load pdf texts

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# loader = PyMuPDFLoader( "./dataset/心臟病知識統整.pdf" )
loader = PyMuPDFLoader( "./dataset/肝病知識統整.pdf" )
pdf_documents = loader.load()

splitter = RecursiveCharacterTextSplitter( chunk_size = 500, chunk_overlap = 50 )
chunks = splitter.split_documents( pdf_documents )

pdf_texts = [ chunk.page_content for chunk in chunks ]
pdf_texts[ 0:2 ]

['欄位 \n值區 \n1 \nage \n20-80 \n2 \ngender \n0=男, 1=女 \n3 \nBMI \n15-40 \n建議 \n過輕：BMI < 18.5 \n●\u200b\n補充營養不挑食，健康增重更有勁！ \n正常：18.5 ≤ BMI < 24.0 \n●\u200b\n吃得剛剛，動得適量，健康體態最理想！ \n過重：24.0 ≤ BMI < 27. \n●\u200b\n少油少糖多運動，健康體態動起來！ \n肥胖：BMI ≥ 27.0 \n●\u200b\n飲食控制搭配運動，健康生活就從今天開始! \n4 \nAlcoholConsumption \n成年男性：每週<14單位的純酒精\u200b\n成年女性：每週<7單位的純酒精 \n一單位=10公克純酒精 \n計算公式：酒的容量(c.c) x酒精濃度(%)x0.785 = 每瓶酒的酒精\n含量(g) \n建議 \n1. 標準飲酒量：男性每週 ≤14 單位, 女性每週 ≤7 單位 \n2. 飲酒過量：男性每週 >14 單位, 女性每週 >7 單位\u200b\n適量飲酒，健康長久；貪杯過量，後患無窮！  \n5 \nsmoking \n0=否, 1=是 \n建議 \n🚭 戒菸一小步，健康一大步！ \n讓呼吸更輕鬆，讓未來更美好！ \n6',
 '0=否, 1=是 \n建議 \n🚭 戒菸一小步，健康一大步！ \n讓呼吸更輕鬆，讓未來更美好！ \n6 \nPhysical Activity \n0-10小時 \n建議 \n每週150分鐘，健康輕鬆擁有！ \n健康從每一次的運動開始，快樂源自於身體的活力！ \n久坐傷身，動起來才青春！ \n7 \nDiabetes \n0=否, 1=是 \n建議 \n正常（70-99 mg/dL） \n●\u200b\n請保持健康生活型態，少糖少油多蔬果，穩定血糖不發愁！ \n糖尿病( >126 mg/dL ) \n●\u200b\n建議立即就醫，按時服藥，及早控制能降低併發症的發生機率。 \n●\u200b\n控制飲食：避免高糖、高碳水化合物食物，均衡攝取蛋白質與健康脂肪。 \n●\u200b\n規律運動：適當運動有助於穩定血糖，提高胰島素敏感性。 \n●\u200b\n定期監測血糖：養成監測血糖的習慣，及早發現異常。  \n8 \nHypertens

pdf texts to embeddings

In [7]:
from gemini_helper import GenAIEmbeddingFunction, to_embeddings

pdf_embeddings = to_embeddings( pdf_texts, type = "RETRIEVAL_DOCUMENT" )

current progress: 0
current progress: 10
len: 13
last index: 15


to parquet file

In [ ]:
import pandas as pd

n = len( pdf_embeddings )
pdf_chunk_ids = [ f"pdf_liver_{i + 1}" for i in range( n ) ]

pdf_df = pd.DataFrame( {
    "id": pdf_chunk_ids[ :n ],
    "document": pdf_texts[ :n ],
    "embedding": pdf_embeddings[ :n ],
    "category": "pdf",
    "domain": "liver_disease",
} )
pdf_df.to_parquet( "liver_disease_data.parquet", engine = "pyarrow", compression = "snappy" )

print( pdf_df.shape )
pdf_df.head( 2 )

(13, 5)


,id,document,embedding,category,domain
0,pdf_liver_1,"欄位 \n值區 \n1 \nage \n20-80 \n2 \ngender \n0=男, ...","[-0.0037563308, 0.005729453, 0.03285417, -0.06...",pdf,liver_disease
1,pdf_liver_2,"0=否, 1=是 \n建議 \n🚭 戒菸一小步，健康一大步！ \n讓呼吸更輕鬆，讓未來更美好...","[0.0013093626, 0.01878251, 0.019596728, -0.062...",pdf,liver_disease


In [12]:
from gemini_helper import GenAIEmbeddingFunction
from chromadb_helper import add_dataframe

pdf_data_df = pd.read_parquet( "liver_disease_data.parquet", engine = "pyarrow" )

chroma_client, collection = add_dataframe( pdf_data_df, GenAIEmbeddingFunction( task_type = "RETRIEVAL_DOCUMENT" ) )
collection.get( limit = 3 ) 

n = 13


{'ids': ['pdf_liver_1', 'pdf_liver_2', 'pdf_liver_3'],
 'embeddings': None,
 'documents': ['欄位 \n值區 \n1 \nage \n20-80 \n2 \ngender \n0=男, 1=女 \n3 \nBMI \n15-40 \n建議 \n過輕：BMI < 18.5 \n●\u200b\n補充營養不挑食，健康增重更有勁！ \n正常：18.5 ≤ BMI < 24.0 \n●\u200b\n吃得剛剛，動得適量，健康體態最理想！ \n過重：24.0 ≤ BMI < 27. \n●\u200b\n少油少糖多運動，健康體態動起來！ \n肥胖：BMI ≥ 27.0 \n●\u200b\n飲食控制搭配運動，健康生活就從今天開始! \n4 \nAlcoholConsumption \n成年男性：每週<14單位的純酒精\u200b\n成年女性：每週<7單位的純酒精 \n一單位=10公克純酒精 \n計算公式：酒的容量(c.c) x酒精濃度(%)x0.785 = 每瓶酒的酒精\n含量(g) \n建議 \n1. 標準飲酒量：男性每週 ≤14 單位, 女性每週 ≤7 單位 \n2. 飲酒過量：男性每週 >14 單位, 女性每週 >7 單位\u200b\n適量飲酒，健康長久；貪杯過量，後患無窮！  \n5 \nsmoking \n0=否, 1=是 \n建議 \n🚭 戒菸一小步，健康一大步！ \n讓呼吸更輕鬆，讓未來更美好！ \n6',
  '0=否, 1=是 \n建議 \n🚭 戒菸一小步，健康一大步！ \n讓呼吸更輕鬆，讓未來更美好！ \n6 \nPhysical Activity \n0-10小時 \n建議 \n每週150分鐘，健康輕鬆擁有！ \n健康從每一次的運動開始，快樂源自於身體的活力！ \n久坐傷身，動起來才青春！ \n7 \nDiabetes \n0=否, 1=是 \n建議 \n正常（70-99 mg/dL） \n●\u200b\n請保持健康生活型態，少糖少油多蔬果，穩定血糖不發愁！ \n糖尿病( >126 mg/dL ) \n●\u200b\n建議立即就醫，按時服藥，及早控制能降低併發症的發生機率。 \n●\u200b\n控制飲食：避免高糖、高碳水化合物食物，均衡攝取蛋白質與健康脂肪。

In [17]:
results = collection.query(
    query_texts = [ "如何降低心血管疾病風險" ], 
    n_results = 3, # how many results to return
    where = { "domain": "heart_disease" },
    include = [ "documents", "metadatas" ]
)
results

NotFoundError: Error getting collection: Collection [546a4a36-a386-41bc-8c64-618e6568b967] does not exists.

In [13]:
from chromadb_helper import delete_collection

delete_collection( chroma_client )